Make data

In [ ]:
from synDataFunctionality.saveSynData import genSynDat
import torch.utils.data as td
from DataLoaders import SynData


In [ ]:
#Test transformations work correctly on dataloader:
from torchvision.transforms import RandomVerticalFlip, RandomHorizontalFlip, Resize, CenterCrop, Normalize
from torchvision.transforms.functional import rotate
import torchvision

t_both = [RandomHorizontalFlip(p=0.5),  RandomVerticalFlip(p=0.5), Resize(size=(640, 640))]
#Maybe normalize imgs automatically in dataSet

In [ ]:
TransDataSet = SynData("SynDat/synInput", "SynDat/synLabel", transforms_both=t_both, repeat_channels=True)

Test huggingface transformers

In [ ]:
from transformers import SegformerForSemanticSegmentation

Test mmseg transformers

In [ ]:
#!mkdir checkpoints
#!git clone https://github.com/open-mmlab/mmsegmentation.git


In [ ]:
import mmseg
import mmcv
print(mmseg.__version__)
print(mmcv.__version__) #skal opdateres til 2.0

In [ ]:
from mmseg.apis import inference_segmentor, init_segmentor, show_result_pyplot
from mmseg.core.evaluation import get_palette


In [ ]:
config_file = 'mmsegmentation/configs/segmenter/segmenter_vit-l_mask_8x1_640x640_160k_ade20k.py'
checkpoint_file = 'checkpoints\segmenter_vit-l_mask_8x1_640x640_160k_ade20k_20220614_024513-4783a347.pth'

config_file_segformer = 'mmsegmentation/configs/segformer/segformer_mit-b5_640x640_160k_ade20k.py'
checkpoint_file_former = 'checkpoints\segformer_mit-b5_640x640_160k_ade20k_20220617_203542-940a6bd8.pth'

In [ ]:
import skimage.io
#model = init_segmentor(config_file, checkpoint_file, device='cpu')
model_segformer = init_segmentor(config_file_segformer, checkpoint_file_former, device='cpu')


In [ ]:
import torch.nn

for param in model_segformer.parameters():
    param.requires_grad = False

model_segformer.decode_head.conv_seg = torch.nn.Conv2d(256, 2, kernel_size=(1, 1), stride=(1, 1))
#model_segformer.mask_norm = torch.nn.LayerNorm((2,), eps=1e-05, elementwise_affine=True)
#model.decode_head.mask_norm = torch.nn.LayerNorm((2,), eps=1e-05, elementwise_affine=True)
print(model_segformer)

#T ODO
#Edit channels in SegmenterMaskTransformerHead
#Edit channels in last TransformerEncoderLayer

In [ ]:
import cv2
img1, lab = TransDataSet[0]
img1 = np.array(img1)
img = np.rollaxis(img1,0,3)

result = inference_segmentor(model_segformer, img)

In [ ]:
print(result[0].shape)

In [ ]:
from torchvision.transforms import ToTensor
print(img.shape)
plt.imshow(img[:, :, 0])
plt.show()

plt.imshow(result[0])
print(np.unique(result[0]))